# GEO Data Organization

This notebook takes in the data files and organizes them into tables to upload to GEO. The tables are organized by Agilent ID and if the data is from a protein or a photoproduct antibody. 


#### (1) Imports and global variables

In [1]:
import os
import shutil

import pandas as pd

GEO_META = "Meta_Data/Meta_Data_GEO.csv"
OUTPUT_FOLDER = "../GEO_Tables"
ALLDATA_FOLER = "../../Data/AllData_Files"

In [2]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

#### (2) Read and display meta data

In [3]:
geo_meta = pd.read_csv(GEO_META)
geo_meta

,GEO_Group,File_ID,GEO_Label
0,Agilent_84702_PhotoProduct_Antibody,CPD_WC_ID42,CPD_NonUV_r1
1,Agilent_84702_PhotoProduct_Antibody,CPD_WC_ID43,CPD_NonUV_r2
2,Agilent_84702_PhotoProduct_Antibody,CPD_UV_ID44,CPD_UV_r1
3,Agilent_84702_PhotoProduct_Antibody,CPD_UV_ID45,CPD_UV_r2
4,Agilent_84702_PhotoProduct_Antibody,64PP_WC_ID46,64PP_NonUV_r1
5,Agilent_84702_PhotoProduct_Antibody,64PP_WC_ID47,64PP_NonUV_r2
6,Agilent_84702_PhotoProduct_Antibody,64PP_UV_ID48,64PP_UV_r1
7,Agilent_84702_PhotoProduct_Antibody,64PP_UV_ID49,64PP_UV_r2
8,Agilent_84702_Proteins,ETS1_WC_ID6,ETS1_NonUV
9,Agilent_84702_Proteins,ETS1_UV_ID7,ETS1_UV


#### (3) Organize Files and Save

In [4]:
for group, group_df in geo_meta.groupby(by="GEO_Group"):
    # Read each file and concatinate into a single df
    print(group)
    # Create subfolder
    group_output = f"{OUTPUT_FOLDER}/{group}"
    os.makedirs(group_output, exist_ok=True)
    dfs = []
    for row in group_df.itertuples():
        # Copy the alldata files and relabel to that in the meta data file
        alldata_file = f"{ALLDATA_FOLER}/{row.File_ID}_alldata.txt"
        agilent_id = row.GEO_Group.split('_')[1]
        shutil.copyfile(alldata_file, f"{group_output}/{row.GEO_Label}_{agilent_id}_alldata.txt")
        # Read the alldata file, remove undefined agilent probes
        df = pd.read_csv(alldata_file, sep='\t')
        df = df.dropna(subset="Sequence").reset_index(drop=True)
        df = df.reset_index(drop=True)
        # Subset the columns to use in the final tables
        signal_label = df.columns[7]
        assert(signal_label == "Alexa488" or signal_label == "Alexa635") 
        df = df[["ID", "Name","Sequence", signal_label]]
        df = df.rename(columns={signal_label: "Signal"})
        df["Dataset"] = row.GEO_Label
        dfs.append(df)
    # Concatinate and pivot to wide format
    result_df = pd.concat(dfs)
    result_df = result_df.pivot(index=["ID", "Name", "Sequence"],
                                columns="Dataset",
                                values="Signal").reset_index()
    # Save as an excel file
    result_df.to_excel(f"{group_output}/{group}.xls", index=False)

Agilent_84702_PhotoProduct_Antibody
Agilent_84702_Proteins
Agilent_85400_Proteins
Agilent_85438_Proteins
Agilent_86684_Proteins
Agilent_86688_PhotoProduct_Antibody
Agilent_86688_Proteins
